In [ ]:
!pip install transformers peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files

In [ ]:
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
import torch
import numpy as np
import pandas as pd
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel
from tqdm import tqdm
import os
import concurrent.futures
from functools import partial

In [ ]:
assert torch.cuda.device_count() == 2

## Configurations

In [ ]:
@dataclass
class Config:
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    lora_dir = '/kaggle/input/lmsys-gemma2-lora-weights/checkpoint-2524_normal/checkpoint-2524'
    max_length = 1776
    batch_size = 4
    device = torch.device("cuda")    
    tta = True
    spread_max_length = False

cfg = Config()

# Load & pre-process Data 

### Stage-1 重复/same case处理

In [ ]:
import numpy as np
import pandas as pd
from functools import partial
import os
import concurrent.futures
from tqdm import tqdm

def adjust_probabilities(prob_a, prob_b, prob_tie, threshold=0.9):
    # 找出最大的概率
    max_prob = max(prob_a, prob_b, prob_tie)
    
    if max_prob >= threshold:
        if max_prob == prob_a:
            return 0.9, 0.05, 0.05
        elif max_prob == prob_b:
            return 0.05, 0.9, 0.05
        else:
            return 0.05, 0.05, 0.9
    else:
        return prob_a, prob_b, prob_tie

def check_duplicates(row, all_data):
    mask_exact = (all_data['prompt'] == row['prompt']) & \
                 (all_data['response_a'] == row['response_a']) & \
                 (all_data['response_b'] == row['response_b'])
    
    mask_swapped = (all_data['prompt'] == row['prompt']) & \
                   (all_data['response_a'] == row['response_b']) & \
                   (all_data['response_b'] == row['response_a'])
    
    exact_matches = all_data[mask_exact]
    swapped_matches = all_data[mask_swapped]
    
    if len(exact_matches) + len(swapped_matches) > 0:
        return swapped_matches, len(swapped_matches) > 0
    
    return None, False

def apply_check_duplicates(row, all_data):
    result, has_swapped = check_duplicates(row, all_data)
    if result is not None:
        # 计算总和
        sum_a = result['winner_model_a'].sum()
        sum_b = result['winner_model_b'].sum()
        sum_tie = result['winner_tie'].sum()
        total = sum_a + sum_b + sum_tie
        
        # 计算概率
        prob_a = sum_a / total
        prob_b = sum_b / total
        prob_tie = sum_tie / total
        
        # 调整概率
        adj_a, adj_b, adj_tie = adjust_probabilities(prob_a, prob_b, prob_tie)
        
        if has_swapped:
            return pd.Series({
                'is_duplicate': True,
                'winner_model_a': adj_b,  # 交换a和b
                'winner_model_b': adj_a,
                'winner_tie': adj_tie
            })
        else:
            return pd.Series({
                'is_duplicate': True,
                'winner_model_a': adj_a,
                'winner_model_b': adj_b,
                'winner_tie': adj_tie
            })
    return pd.Series({'is_duplicate': False, 'winner_model_a': None, 'winner_model_b': None, 'winner_tie': None})

def parallel_check_duplicates(test_chunk, all_data):
    return test_chunk.apply(lambda row: apply_check_duplicates(row, all_data), axis=1)

In [ ]:
# Load and preprocess data
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
# train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
# test_0 = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
# test = test.iloc[0:10]  # 测试功能
# test = pd.concat([test, test_0], axis=0)

In [ ]:
all_data_to_be_check = pd.read_csv('/kaggle/input/all-data-to-be-checked/lmsys-33k.csv')
# all_data_to_be_check_1 = pd.read_csv('/kaggle/input/all-data-to-be-checked/train.csv')
# all_data_to_be_check = pd.concat([all_data_to_be_check_0, all_data_to_be_check_1], axis=0)


In [ ]:
# 处理两个response相同的情况（保持不变）
same_response_mask = test["response_a"] == test["response_b"]
test.loc[same_response_mask, "is_duplicate"] = True
test.loc[same_response_mask, "winner_model_a"] = 0.05
test.loc[same_response_mask, "winner_model_b"] = 0.05
test.loc[same_response_mask, "winner_tie"] = 0.9

In [ ]:
# 并行处理重复检查
print("Checking for duplicates...")
num_cores = 2 # os.cpu_count()  # 获取CPU核心数

# 将test数据分成num_cores份
chunk_size = len(test) // num_cores
test_chunks = [test[i:i+chunk_size] for i in range(0, len(test), chunk_size)]

# 使用partial函数固定all_data_to_be_check参数
partial_check = partial(parallel_check_duplicates, all_data=all_data_to_be_check)

# 使用ProcessPoolExecutor
with concurrent.futures.ProcessPoolExecutor(max_workers=num_cores) as executor:
    results = list(tqdm(executor.map(partial_check, test_chunks), total=len(test_chunks), desc="Processing chunks"))

# 合并结果
duplicate_results = pd.concat(results)

# 更新test数据框
test[['is_duplicate', 'winner_model_a', 'winner_model_b', 'winner_tie']] = duplicate_results

# 创建包含重复和相同响应的submission_df
duplicate_df = test[test['is_duplicate']][["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]

# 准备需要推理的数据
test_for_inference = test[~test['is_duplicate']].copy()

### Stage-2 Inference

# Tokenize

In [ ]:
# 1. 数据准备和重复检验
def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

for df in [test, all_data_to_be_check]:
    for col in ['prompt', 'response_a', 'response_b']:
        df[col] = df[col].apply(process_text)

In [ ]:
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    prompt = ["<prompt>: " + p for p in prompt]
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]
    if spread_max_length:
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        attention_mask = [[1]* len(i) for i in input_ids]
    else:
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
    return input_ids, attention_mask

In [ ]:
# 准备推理数据
tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
tokenizer.add_eos_token = True
tokenizer.padding_side = "right"

data = pd.DataFrame()
data["id"] = test_for_inference["id"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test_for_inference["prompt"], test_for_inference["response_a"], test_for_inference["response_b"])
data["length"] = data["input_ids"].apply(len)

if cfg.tta:
    aug_data = pd.DataFrame()
    aug_data["id"] = test_for_inference["id"]
    aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test_for_inference["prompt"], test_for_inference["response_b"], test_for_inference["response_a"])
    aug_data["length"] = aug_data["input_ids"].apply(len)

# Load model

In [ ]:
import torch.nn as nn

class CustomClassificationHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.fc1 = nn.Linear(config.hidden_size, config.hidden_size, bias=False)
        self.fc2 = nn.Linear(config.hidden_size, config.hidden_size // 2, bias=False)
        self.fc3 = nn.Linear(config.hidden_size // 2, config.num_labels, bias=False)

    def forward(self, features):
        x = self.fc1(features)
        x = self.fc2(x)
        x = self.fc3(x)
        return x



In [ ]:
# Load base model on GPU 0
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False,
    attn_implementation="sdpa",

)
# 替换分类层
model_0.score = CustomClassificationHead(model_0.config)
model_0.score = model_0.score.to(device_0)


# Load base model on GPU 1
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_1,
    use_cache=False,
    attn_implementation="sdpa",

)
# 替换分类层
model_1.score = CustomClassificationHead(model_1.config)
model_1.score = model_1.score.to(device_1)

#### Load LoRA adapter

In [ ]:
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)

# Inference


In [ ]:
from tqdm import tqdm

In [ ]:
@torch.no_grad()
@torch.cuda.amp.autocast()


def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []
    
    # 创建进度条
    progress_bar = tqdm(total=len(df), desc=f"Inference on {device}", unit="sample")
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()
        
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
        
        # 更新进度条
        progress_bar.update(len(tmp))
    
    # 关闭进度条
    progress_bar.close()
    
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df

In [ ]:
# Perform inference
st = time.time()

data = data.sort_values("length", ascending=False)
sub_1 = data.iloc[0::2].copy()
sub_2 = data.iloc[1::2].copy()

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

print(f"Inference elapsed time: {time.time() - st}")

In [ ]:
if cfg.tta:
    st = time.time()
    aug_data = aug_data.sort_values("length", ascending=False)
    sub_1 = aug_data.iloc[0::2].copy()
    sub_2 = aug_data.iloc[1::2].copy()

    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

    tta_result_df = pd.concat(list(results), axis=0)
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    proba = (proba + tta_proba) / 2

    print(f"TTA elapsed time: {time.time() - st}")

In [ ]:
# 5. 合并结果并生成最终的submission_df
mask = proba > 1.0
proba_processed = np.where(mask.any(axis=1)[:, np.newaxis], mask, proba)

result_df.loc[:, "winner_model_a"] = proba_processed[:, 0]
result_df.loc[:, "winner_model_b"] = proba_processed[:, 1]
result_df.loc[:, "winner_tie"] = proba_processed[:, 2]

inference_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]

# 合并重复/相同响应的结果和推理结果
submission_df = pd.concat([duplicate_df, inference_df])

# 保存为 CSV
submission_df.to_csv('submission.csv', index=False)
print("Submission file created.")

In [ ]:
submission_df